# **Hybrid Search LangChain**

In [1]:
!pip install --upgrade --quiet pinecone pinecone-text pinecone-notebooks

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()


api_key=os.getenv("pinecone_api_key")

In [3]:
!pip install langchain-community

In [4]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [5]:
import os
from pinecone import Pinecone,ServerlessSpec
index_name="hybrid-search-langchain-pinecone"
## initialize the Pinecone Client
pc=Pinecone(api_key=api_key)


## create the index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384, # dimension of dense vector
        metric='dotproduct', ## sparse values supported only for dotproduct
        spec=ServerlessSpec(cloud='aws',region='us-east-1')
    )

In [6]:
index=pc.Index(index_name)
index

/Users/pahul17/Documents/Hybrid_Search_RAG/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
!pip install langchain-huggingface sentence-transformers

In [8]:
## vector embedding and sparse matrix
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [9]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder = BM25Encoder().default()
bm25_encoder

In [10]:
sentences = [

    "In 2023, I visited Germany",
    "In 2022, I visited California",
    "In 2021, I visited Paris",
]

## tfidf values on these sentences
bm25_encoder.fit(sentences)


# Storing the values to a json file
bm25_encoder.dump("bm25_values.json")

# load to your BM25Encoder object
bm25_encoder = BM25Encoder().load("bm25_values.json")


100%|██████████| 3/3 [00:00<00:00, 265.96it/s]


In [11]:
retriever=PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index)

In [12]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x14789f050>, index=<pinecone.db_data.index.Index object at 0x10be46d50>)

In [13]:
retriever.add_texts(
    [

    "In 2023, I visited Germany",
    "In 2022, I visited California",
    "In 2021, I visited Paris",
]
)

100%|██████████| 1/1 [00:01<00:00,  1.07s/it]


In [14]:
retriever.invoke("What city did I visit first?")

[Document(metadata={'score': 0.235569701}, page_content='In 2021, I visited Paris'),
 Document(metadata={'score': 0.208107218}, page_content='In 2023, I visited Germany'),
 Document(metadata={'score': 0.206826434}, page_content='In 2022, I visited California')]